In [1]:
from datasets import load_dataset, DatasetDict
import os

dataset = load_dataset("json", data_files="diversevul_20230702.json")

train_valid = dataset["train"].train_test_split(test_size=0.2, seed=0, train_indices_cache_file_name="train.indices")
train_data, valid_data = train_valid["train"], train_valid["test"]
valid_test = valid_data.train_test_split(test_size=0.5, seed=0, train_indices_cache_file_name="valid.indices", test_indices_cache_file_name="test.indices")
valid_data, test_data = valid_test["train"], valid_test["test"]
dataset = DatasetDict({
    "train": train_data,
    "validation": valid_data,
    "test": test_data,
})

dataset["train"]

Dataset({
    features: ['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size', 'message'],
    num_rows: 264393
})

In [2]:
import pandas as pd
df_train = pd.DataFrame(dataset["train"])
#lst_train = pd.DataFrame(dataset["train"]).loc[(dataset['train']['target'] == [1])]['func']
lst_train = df_train.loc[(df_train['target'] == 1)]
lst_train_func = lst_train['func']

In [5]:
lst_train_func = list(lst_train_func)

In [6]:
lst_train_func_new_zet = []
lst_train_func_new_unzet = []
lst_train_func_new = []
i = 0
while i <= 200:
        lst_train_func_new_zet.append(list(lst_train_func)[i])
        lst_train_func_new_zet.append('\n<sep>\n')
        i += 1


In [7]:
with open("ex_rag_vuln_223_pr.txt", "a") as file:
    file.writelines(lst_train_func_new_zet)

In [8]:
import pickle 
from pickle import dump

def load_doc(filename):
    file = open(filename, mode = 'rt', encoding = 'latin-1')
    text = file.read()
    file.close()
    return text
def to_sentences(doc):
    return doc.strip().split('\n<sep>\n')
def sentence_lenghts(sentences):
    lenghts = [len(s.split()) for s in sentences]
    return min(lenghts), max(lenghts)

In [9]:
import re
import string 
import unicodedata
def clean_lines(lines):
    cleaned = list()
    
    for line in lines:
        line = unicodedata.normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('latin-1')
        line = line.split()
        line = [word.lower() for word in line]
        cleaned.append(' '.join(line))
    return cleaned 
        

In [ ]:
# файл расширения pdf нужно сконвертировать из полученного файла txt

In [1]:
filename = 'ex_rag_vuln_223_pr.pdf'
doc = load_doc(filename)
sentences = to_sentences(doc)
for i in range(len(lst_train_func_new_zet)):
    if i%2 == 0:
        sentences.append(lst_train_func_new_zet[i])
minlen, maxlen = sentence_lenghts(sentences)
print('RAG...: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
cleanf = clean_lines(sentences)
filename = 'ex_rag_vuln_223_pr.pkl'
outfile = open(filename, 'wb')
pickle.dump(cleanf, outfile)
outfile.close()
print(filename, 'saved')


In [ ]:
from pickle import load
from pickle import dump
from collections import Counter 
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
def save_clean_sentences(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s'%filename)
    

In [ ]:
def to_vocab(lines):
    vocab = Counter()
    for line in lines:
        tokens = get_tokens(line)
        vocab.update(tokens)
    return vocab
def trim_vocab(vocab, min_occurrence):
    tokens = [k for k,c in vocab.items() if c >= 2]
    return set(tokens)
def update_dataset(lines, vocab):
    new_lines = list()
    for line in lines:
        new_tokens = list()
        for token in line.split():
            if token in vocab:
                new_tokens.append(token)
            else:
                new_tokens.append('unk')
        new_line = ' '.join(new_tokens)
        new_lines.append(new_line)
    return new_lines


In [ ]:
def get_tokens(data):
    data = data.replace('\r', '')
    data = data.replace('\t', ' ')
    data = data.lower()
    
    re_fun = re.compile(r'([_.->\w]+)\s*\([^)]*\)')
    re_var = re.compile(r'\s*([_\d\w]*)(\s*)=(\s*)[^;]*')

    var_idx = 0 
    func_idx = 0
    words = []
    for line in data.split('\n'):

        s = re_var.search(line)
        if s:
            var_idx += 1
            var_name = s.group(1)
            line = re.compile(r'([_\d\w]+\s*=\s*)').sub('var%d = '% var_idx, line)

        s = re_fun.search(line)
        if s:
            got_stmt = False
            for st in ['if','for','while']:
                if line.find(st) >=0:
                    got_stmt = True
                    break
            if not got_stmt:
                func_idx += 1
                func = s.group(1)
                line = line.replace('%s' % func, 'func%d' % func_idx)

        chars = ['(', ')', '{', '}', '*', '/', '+', '-', '=', ';', ',']
        for ch in chars:
            line = line.replace(ch, ' %s ' % ch)

        if line and len(line) >= 1:         
            for w in line.split(' '):
                w = w.strip()
                if w:
                    words.append(w) 

    return words




In [ ]:
import fitz 
from tqdm.auto import tqdm  

def text_formatter(text: str) -> str:
    
    cleaned_text = text.replace("\n", " ").strip() 
    
    return cleaned_text


In [ ]:
def open_and_read_pdf_2(pdf_path: str) -> list[dict]: #!!!
    with open('ex_rag_vuln_223_pr.txt', 'r') as file:
        docs = file.read()
    dc= docs.split('<sep>')
    doc = fitz.open('ex_rag_vuln_223_pr.pdf') 
    p_and_t = []
    a = 0
    i = 0
    for page_number, page in tqdm(enumerate(doc)): 
        while i <=402:
            text = page.get_text()  
            text = text_formatter(text)
            p_and_t.append({"page_number": page_number,  
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split("<sep>")),
                                "page_token_count": len(text) / 4,
                                 "text": dc[i]})
            i += 1 
    return p_and_t
pages_and_texts_2 = open_and_read_pdf_2(pdf_path='ex_rag_vuln_223.pdf')

In [ ]:
filename = 'ex_rag_vuln_223_pr.pkl'
lines1 = load_clean_sentences(filename)
vocab = to_vocab(lines1)
print('rag Vocabulary: %d' % len(vocab))
vocab = trim_vocab(vocab, 5)
print('New rag Vocabulary: %d' % len(vocab))
#lines1 = update_dataset(lines1, vocab)
filename = 'rag_vocab_vuln_223_pr.pkl'
save_clean_sentences(lines1, filename)
print(len(lines1))
for i in range(34,70): #example
    print("line",i,":",lines1[i])

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts_2)

In [ ]:
import sys
a = 0
for item in tqdm(pages_and_texts_2):
    item["sentences"] = str(item["text"])
    
    item["page_sentence_count_spacy"] = len(item["sentences"])
    


In [ ]:
df = pd.DataFrame(pages_and_texts_2)

In [ ]:
num_sentence_chunk_size = 100000000 #random number +-

def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts_2):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    
    item["num_chunks"] = len(item["sentence_chunks"])

In [ ]:
df = pd.DataFrame(pages_and_texts_2)

In [ ]:
import re

pages_and_chunks_2 = []

for item in tqdm(pages_and_texts_2):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks_2.append(chunk_dict)


len(pages_and_chunks_2)
df = pd.DataFrame(pages_and_chunks_2)

In [ ]:
embedding_model.to("cuda") 
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cuda") 

for item in tqdm(pages_and_chunks_2):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [ ]:
text_chunks_2 = [item["sentence_chunk"] for item in pages_and_chunks_2]

In [ ]:
text_chunk_embeddings_2 = embedding_model.encode(text_chunks_2,
                                               batch_size=32, 
                                               convert_to_tensor=True)

In [ ]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_2)
embeddings_df_save_path = "rag_chunks_and_vuln_df_2_pr.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)

In [3]:
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("rag_chunks_and_vuln_df_2_pr.csv")

text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

FileNotFoundError: [Errno 2] No such file or directory: 'rag_chunks_and_vuln_df_2_pr.csv'

In [ ]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) 

In [ ]:
import textwrap

def print_wrapped(text, wrap_length=200):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)
import torch


In [ ]:
def relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 
    
    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top(query: str,
            embeddings: torch.tensor,
            pages_and_chunks: list[dict]=pages_and_chunks,
            n_resources_to_return: int=5):
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    
    print("Results:")
    
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        
        print("\n")